In [ ]:
import mysql.connector

conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark")

cursor = conn.cursor()
import pandas as pd

In [ ]:
create_customer_table = """CREATE TABLE customer (
    customerkey INT PRIMARY KEY,
    gender VARCHAR(10),
    name VARCHAR(100),
    city VARCHAR(100),
    state_code VARCHAR(100),
    state VARCHAR(100),
    zip_code VARCHAR(20),
    country VARCHAR(100),
    continent VARCHAR(50),
    birthday DATE,
    age INT,
    segment VARCHAR(50)
);"""
cursor.execute(create_customer_table)
conn.commit()

In [ ]:
create_exchange_table = """CREATE TABLE exchange_rates (
    date DATE NOT NULL,
    currency VARCHAR(10) NOT NULL,
    exchange DECIMAL(10, 4) NOT NULL
);"""
cursor.execute(create_exchange_table)
conn.commit()

In [ ]:
create_product_table = """CREATE TABLE product (
    productkey INT PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    brand VARCHAR(255) NOT NULL,
    color VARCHAR(50),
    unit_cost_usd DECIMAL(10, 2),
    unit_price_usd DECIMAL(10, 2),
    subcategorykey INT,
    subcategory VARCHAR(255),
    categorykey INT,
    category VARCHAR(255)
);"""
cursor.execute(create_product_table)
conn.commit()

In [ ]:
create_sales_table='''CREATE TABLE sales (
    order_number INT,
    line_item INT NOT NULL,
    order_date DATE NOT NULL,
    delivery_date DATE,
    customerkey INT NOT NULL,
    storekey INT NOT NULL,
    productkey INT NOT NULL,
    quantity INT NOT NULL,
    currency_code VARCHAR(10),
    shipping_days INT
);'''
cursor.execute(create_sales_table)
conn.commit()

In [ ]:
create_store_table = """CREATE TABLE stores (
    storekey INT PRIMARY KEY,
    country VARCHAR(100),
    state VARCHAR(100),
    square_meters DECIMAL(10, 2),
    open_date DATE
);"""
cursor.execute(create_store_table)
conn.commit()

In [ ]:
create_calendar_table = """CREATE TABLE calendar (
    date DATE PRIMARY KEY
);"""
cursor.execute(create_calendar_table)
conn.commit()

In [ ]:
merge_table = """select s.*, er.exchange, st.*,p.*,c.*
from sales s
join exchange_rates er 
	on (s.order_date = er.date and s.currency_code = er.currency)
join stores st
	on st.storekey = s.storekey
join product p
	on p.productkey = s.productkey
join customer c
	on c.customerkey = s.customerkey;"""

In [ ]:
combined_sales = """create table combined_sales as
select s.*, er.exchange, 
st.country as store_country,
st.state as store_state,
st.square_meters,
st.open_date,
p.product_name,
p.brand as brand,
p.color,
p.unit_cost_usd,
p.unit_price_usd,
p.subcategorykey,
p.subcategory,
p.categorykey,
p.category,
c.gender,
c.name as customer_name,
c.city,
c.state_code,
c.state,
c.zip_code,
c.country,
c.continent,
c.birthday,
c.age,
c.segment
from sales s
join exchange_rates er 
	on (s.order_date = er.date and s.currency_code = er.currency)
join stores st
	on st.storekey = s.storekey
join product p
	on p.productkey = s.productkey
join customer c
	on c.customerkey = s.customerkey;"""

In [ ]:
select_query = """SELECT * FROM combined_sales;"""
cursor.execute(select_query)
res = conn.commit()

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
columns = [
    'order_number', 'line_item', 'order_date', 'delivery_date', 'customerkey',
    'storekey', 'productkey', 'quantity', 'currency_code', 'shipping_days',
    'exchange', 'store_country', 'store_state', 'square_meters', 'open_date',
    'product_name', 'brand', 'color', 'unit_cost_usd', 'unit_price_usd',
    'subcategorykey', 'subcategory', 'categorykey', 'category', 'gender',
    'customer_name', 'city', 'state_code', 'state', 'zip_code', 'country',
    'continent', 'birthday', 'age', 'segment'
]

# Create DataFrame with the fetched results and the defined column names
dft = pd.DataFrame(res, columns=columns)
dft.head(5)